In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Lastsemester/Social_Network_Analysis/project

Mounted at /content/gdrive
/content/gdrive/My Drive/Lastsemester/Social_Network_Analysis/project


In [1]:
# Importing modules
import pandas as pd
import numpy as np

In [2]:
#! pip install transformers

from transformers import AutoModel
from transformers import AutoTokenizer
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
#print(f"Input tensor shape: {inputs['input_ids'].size()}")

In [4]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]), hidden_states=None, attentions=None)


In [5]:
output_hidden = outputs.last_hidden_state
output_hidden.shape

torch.Size([1, 6, 768])

# Practice Dataset

In [11]:
product = pd.read_csv("data/product_catalogue-v0.2.csv")
product.shape

(883868, 7)

In [6]:
d = {'product_id': ['B0188A3QRM', 'B075VXJ9VG'], 
     'product_title': ['Amazon Basics Woodcased #2 Pencils, Unsharpened', 'BAZIC Pencil #2 HB Pencils, Latex Free Eraser'],
     'product_description': [np.NaN, '<p><strong>BACK TO BAZIC</strong></p><p>Our go'], 
     'product_bullet_point': ['144 woodcase #2 HB pencils made from high-quality', '&#11088; UN-SHARPENED #2 PREMIUM PENCILS.'],
     'product_brand': ['Amazon Basics	', 'BAZIC Products	'],
     'product_color_name': [ 'Yellow', '12-count'],
     'product_locale': [ 'us', 'us']}
df = pd.DataFrame(data=d)
df

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened",NaN,144 woodcase #2 HB pencils made from high-quality,Amazon Basics\t,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser",<p><strong>BACK TO BAZIC</strong></p><p>Our go,&#11088; UN-SHARPENED #2 PREMIUM PENCILS.,BAZIC Products\t,12-count,us


In [12]:
# add ".astype(str)" 
df = product
df['Combined'] = df['product_title'].astype(str) + ', ' + df['product_description'].astype(str) + ', ' + df['product_bullet_point'].astype(str)  + ', ' + df['product_brand'].astype(str) + ', ' + df['product_color_name'].astype(str) + ', ' + df['product_locale'].astype(str)
df = pd.DataFrame(data=df)
df

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,Combined
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpene...",NaN,144 woodcase #2 HB pencils made from high-qual...,Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpene..."
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,...",<p><strong>BACK TO BAZIC</strong></p><p>Our go...,&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each...,BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,..."
2,B07G7F6JZ6,Emraw Pre Sharpened Round Primary Size No 2 Ju...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 8 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us,Emraw Pre Sharpened Round Primary Size No 2 Ju...
3,B07JZJLHCF,Emraw Pre Sharpened Triangular Primary Size No...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 6 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us,Emraw Pre Sharpened Triangular Primary Size No...
4,B07MGKC3DD,"BIC Evolution Cased Pencil, #2 Lead, Gray Barr...",NaN,Premium #2 HB lead pencils with break-resistan...,Design House,Gray,us,"BIC Evolution Cased Pencil, #2 Lead, Gray Barr..."
...,...,...,...,...,...,...,...,...
883863,B094NN6T8V,アイリスプラザ コーナーソファ 3点セット 高反発 2人掛け 3人掛け ブラウン 394310,NaN,商品サイズ（cm）：幅約154×奥行約109×高さ約33（※組み合わせ時）\n商品重量：約6...,アイリスプラザ(IRIS PLAZA),ブラウン,jp,アイリスプラザ コーナーソファ 3点セット 高反発 2人掛け 3人掛け ブラウン 39431...
883864,B096RD95K9,Uping テーブル・ベッドの高さ調節が簡単にできる ベッドの高さをあげる足 4個セット 高...,B096RD95K9,サイズ:約16.5×11.7×8.2cm\n総重量:約1.23kg\n材質:ABSプラスチッ...,Uping,NaN,jp,Uping テーブル・ベッドの高さ調節が簡単にできる ベッドの高さをあげる足 4個セット 高...
883865,B099Z4HXH1,アイリスプラザ ソファ ソファー ローソファ ローソファー 1人掛け 1人用 幅約108cm...,NaN,商品サイズ(cm):幅約108×奥行約44×高さ約62\n商品重量:約15kg\n材質 表面...,アイリスプラザ(IRIS PLAZA),グレー,jp,アイリスプラザ ソファ ソファー ローソファ ローソファー 1人掛け 1人用 幅約108cm...
883866,B09B9QMMSF,足入れヒーター 足温器 電気足温器 1～6時間タイマー 丸洗い可能 省エネ あったか脚入れヒ...,NaN,【腰からスッポリ包む、一人様用のごたつ】 つま先・腰・お腹まですっぽり包む寝袋のようなデザイ...,タサイスク,ブラウン,jp,足入れヒーター 足温器 電気足温器 1～6時間タイマー 丸洗い可能 省エネ あったか脚入れヒ...


# Running the model on practice dataset

Running on only first string

In [84]:
s = df['Combined']

text = s[9][:2000]
print(text)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs.last_hidden_state.shape) 

YOTINO Pre-sharpened Wood Cased #2 HB Pencils - Box of 100, </br>YOTINO Pre-sharpened #2 HB Pencils Pack of 100 </br> </br>Description </br>These HB pencils in bulk are perfect for restocking the writing materials in your office or classroom. Plus, if you’re an artist who frequently sketches, these are great to have. </br> </br>Whether you’re taking notes, making unique artwork, or sketching out a custom design, this is the best choice for premium quality pencils. </br> </br> </br>Feature: </br>Break-Resistant </br> The dense graphite cores make these pencils ideal for drawing, writing & sketching </br> </br>Durable Latex-Free Erasers </br>Will erase your marks without damaging your paper </br> </br>Sharpened & Ready-to-Use </br>Each pencil comes pre-sharpened, You can start writing, drawing, or sketching as soon as you open the box. </br> </br>Convenient Bulk Pack </br>You’ll always have enough pencils for your office, classroom or home . </br> </br>Package Includes: </br>Pre-sharpene

IndexError: index out of range in self

Looping through now

In [67]:
l = []
for text in df['Combined'][4:10]:
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        print(outputs.last_hidden_state.shape)
        #l.append(outputs.last_hidden_state)

torch.Size([1, 96, 768])
torch.Size([1, 112, 768])
torch.Size([1, 101, 768])
torch.Size([1, 86, 768])
torch.Size([1, 442, 768])


IndexError: index out of range in self

In [68]:
for i in df["Combined"][4:10]:
    print(i)

BIC Evolution Cased Pencil, #2 Lead, Gray Barrel, 24-Count (PGEBP241-BLK), nan, Premium #2 HB lead pencils with break-resistant lead and splinter-free barrels
Durable, long-lasting leads and erasers
Latex-free erasers easily erase stray marks
Trendy and modern gray barrels
Pack of 24 certified non-toxic pencils ideal for home or classroom, Design House, Gray, us
Maped Essentials Triangular Graphite #2 Pencils, Pack of 12 (851779ZT), nan, Classic Yellow #2 Graphite Pencils with a modern twist!
High quality #2 / HB Graphite is break resistant & goes down smooth!
Triangular shape is more comfortable to hold and easier to use than a standard pencil, will not roll and is recommended by teachers!
Sharpen easily in any standard pencil sharpener
Customer will receive 12 pre-sharpened Graphite Pencils, Maped Helix USA, Yellow, us
BIC Xtra Fun Cased Pencil, 2 Lead, Assorted Barrel Colors, 48-Count, nan, Kid-friendly pencils feature bright two-toned colored barrels to make writing fun
Smooth, bre